In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
import json
import io
# Install Kaggle library
!pip install -q kaggle
import os
os.environ['KAGGLE_USERNAME'] = "comp540jx24" # username from the json file
os.environ['KAGGLE_KEY'] = "ff43bff394af31725e0e8929b78ee4da" # key from the json file
!kaggle competitions download -c ifood-2019-fgvc6 # api copied from kaggle
!unzip train_set.zip
!unzip val_set.zip
!unzip test_set.zip
!unzip train_labels.csv.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_set/test_009481.jpg  
  inflating: test_set/test_013960.jpg  
  inflating: test_set/test_002362.jpg  
  inflating: test_set/test_005803.jpg  
  inflating: test_set/test_015388.jpg  
  inflating: test_set/test_002764.jpg  
  inflating: test_set/test_021964.jpg  
  inflating: test_set/test_013738.jpg  
  inflating: test_set/test_016448.jpg  
  inflating: test_set/test_022310.jpg  
  inflating: test_set/test_002203.jpg  
  inflating: test_set/test_003963.jpg  
  inflating: test_set/test_022431.jpg  
  inflating: test_set/test_003303.jpg  
  inflating: test_set/test_006227.jpg  
  inflating: test_set/test_016860.jpg  
  inflating: test_set/test_009457.jpg  
  inflating: test_set/test_020554.jpg  
  inflating: test_set/test_003655.jpg  
  inflating: test_set/test_006939.jpg  
  inflating: test_set/test_003241.jpg  
  inflating: test_set/test_020979.jpg  
  inflating: test_set/test_027004.jpg  
  inflating: test_set/test_02

In [0]:
import gc 
gc.collect()

15

In [0]:
! pip install pretrainedmodels

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=03d3726200bbc15fb74ecda15ee96a11800c22a7b09eaa91c786e658c46a3908
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [0]:
import numpy as np
import pandas as pd
from matplotlib import style
import seaborn as sns

style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
from sklearn.utils import shuffle
import pretrainedmodels

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train_dir = 'train_set/'
val_dir = 'val_set/'
train_df = pd.read_csv('train_labels.csv')
val_df = pd.read_csv('val_labels.csv')
train_df['path'] = train_dir+ train_df['img_name']
val_df['path']  =  val_dir+ val_df['img_name']
df = pd.concat([train_df, val_df], ignore_index=True)
df.head()
val_idx = [i for i in range(len(train_df), len(df))]

In [0]:
sz = 256
bs = 128

In [0]:
#np.random.seed(142)
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      max_warp=0.1,
                      max_rotate = 20,
                      max_zoom=1.4,
                      max_lighting=0.2,
                      p_lighting=0.2, 
                      p_affine=0.4
                      ) #change as needed
src = (ImageList.from_df(df=df,path='./',cols='path') #get dataset
       .split_by_idx(val_idx)
       .label_from_df(cols='label') #obtain labels
      )
src = (ImageList.from_df(df=df,path='./',cols='path') #get dataset from dataset
       .split_by_idx(val_idx)
        .label_from_df(cols='label') #obtain labels from the level column
      )
data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='border') #Data augmentation
        .databunch(bs=bs,num_workers=4) #DataBunch
        .normalize(imagenet_stats) #Normalize
       )

In [0]:
gc.collect()

0

In [0]:
def top_3(preds, targs):
    return top_k_accuracy(preds, targs, 3)

## ** Uploading the Learners**

In [0]:
#RESNEXT101

def get_cadene_model(pretrained=True, model_name='se_resnext101_32x4d'):
    if pretrained:
        arch = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    else:
        arch = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=None)
    return arch

learner1 = cnn_learner(data, get_cadene_model, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy()).mixup()
learner1 = learner1.load('/content/drive/My Drive/Colab Notebooks/stage-1-Resnext101')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


In [0]:
learner1.validate()

[1.8644637, tensor(0.7193), tensor(0.8802)]

In [0]:
#RESNET152
learner2 = cnn_learner(data, models.resnet152, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy())
learner2 = learner2.load('/content/drive/My Drive/Colab Notebooks/experimental_stage-8-Resnet152')

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth


In [0]:
learner2.validate()

[1.9503111, tensor(0.7077), tensor(0.8671)]

In [0]:
#RESNET101
learner3 = cnn_learner(data, models.resnet101, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy())
learner3 = learner3.load('/content/drive/My Drive/Colab Notebooks/stage-6_Resnet101')

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [0]:
learner3.validate()

[1.9974058, tensor(0.7002), tensor(0.8686)]

In [0]:
#RESNET50
learner4 = cnn_learner(data, models.resnet50, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy())
learner4 = learner4.load('/content/drive/My Drive/Colab Notebooks/stage-experimental_top_3_Resnet50')

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
learner4.validate()

[2.3462112, tensor(0.6680), tensor(0.8418)]

In [0]:
'''#RESNET34
learner5 =  cnn_learner(data, models.resnet34, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy()).mixup()
learner5 = learner5.load('/content/drive/My Drive/Colab Notebooks/stage-experimental_Resnet34')'''

"#RESNET34\nlearner5 =  cnn_learner(data, models.resnet34, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy()).mixup()\nlearner5 = learner5.load('/content/drive/My Drive/Colab Notebooks/stage-experimental_Resnet34')"

In [0]:
'''learner5.validate()'''

'learner5.validate()'

In [0]:
'''#RESNET18
learner6 =  cnn_learner(data, models.resnet18, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy()).mixup()
learner6 = learner6.load('/content/drive/My Drive/Colab Notebooks/stage-experimental_Resnet18')'''

"#RESNET18\nlearner6 =  cnn_learner(data, models.resnet18, metrics=[accuracy, top_3],loss_func = LabelSmoothingCrossEntropy()).mixup()\nlearner6 = learner6.load('/content/drive/My Drive/Colab Notebooks/stage-experimental_Resnet18')"

Experiments

In [0]:
preds1, _ = learner1.get_preds(ds_type=DatasetType.Valid)
preds2, _ = learner2.get_preds(ds_type=DatasetType.Valid)
preds3, _ = learner3.get_preds(ds_type=DatasetType.Valid)
preds4, _ = learner4.get_preds(ds_type=DatasetType.Valid)
'''preds5, _ = learner5.get_preds(ds_type=DatasetType.Valid)
preds6, _ = learner6.get_preds(ds_type=DatasetType.Valid)'''

'preds5, _ = learner5.get_preds(ds_type=DatasetType.Valid)\npreds6, _ = learner6.get_preds(ds_type=DatasetType.Valid)'

In [0]:
#pred1 =   preds1 +0.85 * preds2 +0.83 * preds3+0.6 * preds4  = (0.8963)
# preds1 +0.55 * preds2 +0.63 * preds3+0.1*preds4
pred1 = preds1 +0.58 * preds2 +0.61 * preds3+0.21*preds4

In [0]:
#pred2 = (preds1 + preds2 + preds3+ preds4)/2

In [0]:
targets = torch.tensor(val_df.label.values)

In [0]:
top_3(pred1, targets)

tensor(0.9013)

In [0]:
#old predition as a reference top_3(pred1, targets)

# Prediction

In [0]:
test = ImageList.from_folder('test_set/')
data.add_test(test)

In [0]:
#Learner1 
learner1.data = data
preds1, _ = learner1.get_preds(ds_type=DatasetType.Test)

In [0]:
fnames1 = [f.name for f in learner1.data.test_ds.items]
fnames1[:4]

['test_007588.jpg', 'test_003586.jpg', 'test_014899.jpg', 'test_003058.jpg']

In [0]:
#Learner2 
learner2.data = data
preds2, _ = learner2.get_preds(ds_type=DatasetType.Test)

In [0]:
fnames2 = [f.name for f in learner2.data.test_ds.items]
fnames2[:4]

['test_007588.jpg', 'test_003586.jpg', 'test_014899.jpg', 'test_003058.jpg']

In [0]:
#Learner3 
learner3.data = data
preds3, _ = learner3.get_preds(ds_type=DatasetType.Test)

In [0]:
fnames3 = [f.name for f in learner3.data.test_ds.items]
fnames3[:4]

In [0]:
#Learner4 
learner4.data = data
preds4, _ = learner4.get_preds(ds_type=DatasetType.Test)

In [0]:
fnames4 = [f.name for f in learner4.data.test_ds.items]
fnames4[:4]

In [0]:
#Weighting
pred = preds1 +0.58 * preds2 +0.61 * preds3+0.21*preds4

In [0]:
pred.argmax(axis = 1)

# Make data frame

In [0]:
col = ['img_name']
test_df = pd.DataFrame(fnames1, columns=col)
test_df['label'] = ''

In [0]:
predictions = np.array(pred).reshape(len(pred), 251)
predictions.shape

In [0]:
from tqdm import tqdm_notebook as T
for i, pred in T(enumerate(predictions), total=len(predictions)):
    test_df.loc[i, 'label'] = ' '.join(str(int(i)) for i in np.argsort(pred)[::-1][:3])

In [0]:
df_example = pd.read_csv('ifood2019_sample_submission.csv')
test_df = pd.merge(df_example['img_name'], test_df )

In [0]:
test_df.to_csv('/content/drive/My Drive/Colab Notebooks/top_3_ensamble_final.csv', index=False)

In [0]:
test_df